In [1]:
#using Pkg
#Pkg.add("Pandas")
using Pandas

In [2]:
dataset = read_csv("ecma_data.csv")

,Subject,G1P1,G1P2,G1P3,G1P4,G2P1,G2P2,G2P3,G2P4
0,1,1,3,2,3,1,1,2,3
1,2,2,3,2,2,2,3,3,3
2,3,2,1,1,1,1,2,2,2
3,4,3,1,2,2,1,2,2,3
4,5,3,2,2,1,3,1,1,3
5,6,1,2,2,1,3,2,2,3
6,7,1,2,1,1,1,2,3,3
7,8,1,2,1,1,1,2,3,3
8,9,1,2,1,1,1,2,3,3
9,10,1,1,1,1,1,2,1,3


In [3]:
length(query(dataset, :(G1P1==1&G1P2==2&G1P3==1&G1P4==1)))

62

In Game 1 every level of player other than level 0 chooses the profile of actions `1,2,1,1`.  Of the 80, 62 chose this profile.  This could randomly include some level 0 players.

Estimate the probability $\rho$ that a randomly selected player from this dataset will have level $0$,  To do this, let $\pi$ be the probability that a randomly selected player will choose the profile `1,2,1,1`.  Then $(1-\rho)$ is the probability he or she won't.  Since 
$$\pi = (1-\rho)+\rho (\frac{1}{3})^4$$ we could estimate $\pi$ then calculate an estimate of $\rho$ from that.

For any $\pi$, the probability that exacly 62 of the 80 players choose the profile `1,2,1,1` is given by
$$\frac{80!}{62!18!}\pi^{62}(1-\pi)^{18}$$

In [4]:
using SymPy
pi,rho = symbols("pi, rho", positive=true)
prob = (pi^(62))*(1-pi)^(18)

 62        18
π  ⋅(1 - π)  

In [5]:
dprob = diff(prob,pi)

      62        17       61        18
- 18⋅π  ⋅(1 - π)   + 62⋅π  ⋅(1 - π)  

In [6]:
solve(dprob,pi)

2-element Vector{Sym}:
 31/40
     1

Using entry 2, the value for $pi$ that maximizes the probability of seeing this data is $\frac{62}{80}$

Now to estimate $\rho$:

In [7]:
solve((62/80)-(1-rho)-rho*(1/3)^4,rho)

1-element Vector{Sym}:
 0.227812500000000

# Both Games

Now we calculate profiles:
1. level 1: `1,2,1,1,1,2,1,3`
2. level 2: `1,2,1,1,1,2,2,3`
3. level 3: `1,2,1,1,1,1,2,3`

K level reasoning only explains 20% of the data.

In [8]:
l1 = length(query(dataset, :(G1P1==1&G1P2==2&G1P3==1&G1P4==1&G2P1==1&G2P2==2&G2P3==1&G2P4==3)))
l2 = length(query(dataset, :(G1P1==1&G1P2==2&G1P3==1&G1P4==1&G2P1==1&G2P2==2&G2P3==2&G2P4==3)))
l3 = length(query(dataset, :(G1P1==1&G1P2==2&G1P3==1&G1P4==1&G2P1==1&G2P2==1&G2P3==2&G2P4==3)))
string(l1,"  ",l2,"  ",l3)

"7  12  14"

In [9]:
l1+l2+l3/80

19.175

Now we can estimate the probabilities with which a randomly drawn player is level 1 as $\rho_1$, or level 2 as $\rho_2$, of level 3 or higher as $\rho_3$.

In [10]:
rho_1,rho_2,rho_3,pi_1,pi_2,pi_3 = symbols("rho_1,rho_2,rho3,pi_1,pi_2,pi_3", positive=true)

(rho_1, rho_2, rho3, pi_1, pi_2, pi_3)

The level 1 profile of actions is player with probability
$$
\pi_1 = \rho_1+(1-\rho_1)(\frac{1}{3})^8
$$
Similary for
$$
\pi_2 = \rho_2+(1-\rho_2)(\frac{1}{3})^8
$$
and
$$
\pi_3 = \rho_3+(1-\rho_3)(\frac{1}{3})^8
$$

The probability of seeing the data is 
$$
\pi_1^7*\pi_2^{12}*\pi_3^{14}*(1-\pi_1-\pi_2-\pi_3)^{80-7-12-14}
$$

# First try: symbolic maximization

Choose values for $\pi_1$,$\pi_2$ and $\pi_3$ to maximize the probability of the data using the above formula.  This method is not a good one and probably won't work.

In [11]:
prob_full = pi_1^7*pi_2^12*pi_3^14*(1-pi_1-pi_2-pi_3)^(80-33)

  7   12   14                    47
π₁ ⋅π₂  ⋅π₃  ⋅(-π₁ - π₂ - π₃ + 1)  

In [12]:
dprob_full = [diff(prob_full,v) for v in [pi_1,pi_2,pi_3]]

3-element Vector{Sym}:
  -47*pi_1^7*pi_2^12*pi_3^14*(-pi_1 - pi_2 - pi_3 + 1)^46 + 7*pi_1^6*pi_2^12*pi_3^14*(-pi_1 - pi_2 - pi_3 + 1)^47
 -47*pi_1^7*pi_2^12*pi_3^14*(-pi_1 - pi_2 - pi_3 + 1)^46 + 12*pi_1^7*pi_2^11*pi_3^14*(-pi_1 - pi_2 - pi_3 + 1)^47
 -47*pi_1^7*pi_2^12*pi_3^14*(-pi_1 - pi_2 - pi_3 + 1)^46 + 14*pi_1^7*pi_2^12*pi_3^13*(-pi_1 - pi_2 - pi_3 + 1)^47

This is the method that you can try, it will probably run on and on without finding a solution.  We can just find values for the $\pi_i$ so that all the derivated are zero.

In [13]:
#solve(dprob_full,pi_1,pi_2,pi_3)

# Method 2 - Numerical

Write down a traditional likelihood function.  It is just the natural log of the `prob_full` symbolic function converted into a function that contains no symbolic stuff.  

In [67]:
llikelihood(x) = (-1)*(7*log(x[1])+12*log(x[2])+14*log(x[3])+(80-7-12-14)*log(1-x[1]-x[2]-x[3]))


llikelihood (generic function with 1 method)

In [68]:
llikelihood([.3,.3,.1])

111.69839638742842

In [69]:
import Pkg
for package in ["BlackBoxOptim", "Distributions", "ForwardDiff", "JSON", "Optim", "Quadrature", "StatsPlots",
    "DotEnv","MySQL","DBInterface","Tables","HTTP", "JSON"]
    if false
        Pkg.add(package)
    end
end
using Optim

In [70]:
l = [0.05,0.05,0.05]
u = [0.95,0.95,0.95]
x0 = [.3,.3,.1]
result = optimize(llikelihood,l,u,x0,Fminbox())

 * Status: success

 * Candidate solution
    Final objective value:     8.921814e+01

 * Found with
    Algorithm:     Fminbox with L-BFGS

 * Convergence measures
    |x - x'|               = 2.01e-10 ≰ 0.0e+00
    |x - x'|/|x'|          = 8.16e-10 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 7.04e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    4
    f(x) calls:    46
    ∇f(x) calls:   46


In [71]:
parameters = Optim.minimizer(result)

3-element Vector{Float64}:
 0.08750000011750786
 0.15000000003539343
 0.17500000000215135

Now we have the estimates for the three $pi$, `.0875`, `.15` and `.175` (all give of take a bit).  As you should probably realize, `.0875` is just $7/80$, so this just illustrates the method.  Now we can find the estimates for the $\rho$.

In [73]:
solve(rho_1+(1-rho_1)*(1/3)^8-parameters[1])

1-element Vector{Sym}:
 0.0873608995077697